In [2]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

# Problem parameters
n = 5  # Number of decision variables
alpha = 1.0  # Fairness parameter, change this to test different alpha
A = torch.tensor([[1.0, 2.0, 3.0, 4.0, 5.0]])  # Knapsack constraint matrix
b = torch.tensor([10.0])  # Knapsack budget

# Define decision variables
x = cp.Variable(n, nonneg=True)  # Non-negative decision variables
t = cp.Variable(n)  # Auxiliary variables for epigraph trick

# Define the cost function (Objective)
if alpha == 1:
    # Logarithmic utility function: max sum_i log(x_i)
    constraints = [t[i] <= cp.log(x[i]) for i in range(n)]
else:
    # Power utility function: max sum_i (x_i^(1 - alpha)) / (1 - alpha)
    constraints = [t[i] * (1 - alpha) <= cp.power(x[i], 1 - alpha) for i in range(n)]

# Knapsack constraint: A * x <= b
constraints.append(A.numpy() @ x <= b.numpy())

# Define the objective: maximize sum_i t_i (linearized objective)
objective = cp.Maximize(cp.sum(t))

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Check if the problem is DCP-compliant (Disciplined Convex Programming)
assert problem.is_dcp(), "The problem is not DCP-compliant. Check constraints."

# Create the CVXPY Layer
cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])

# Test the layer with input parameters (A, b)
A_input = torch.tensor([[1.0, 2.0, 3.0, 4.0, 5.0]], requires_grad=True)
b_input = torch.tensor([10.0], requires_grad=True)

# Solve the optimization problem as part of a differentiable pipeline
solution, = cvxpylayer(A_input, b_input)
print("Optimal solution:", solution)

# If needed, compute gradients with respect to parameters A and b
solution.sum().backward()
print("Gradient with respect to A:", A_input.grad)
print("Gradient with respect to b:", b_input.grad)


ValueError: The layer's parameters must exactly match problem.parameters